In [1]:
#!pip install textsearch
#import nltk
#import contractions
#import inflect
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize
#from nltk.corpus import stopwords
#from textsearch import TextSearch

import pandas as pd
import re, string, unicodedata
from bs4 import BeautifulSoup

In [2]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8-sig', 'ignore')
    str = re.sub("b'|b\"",'',str)
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove_laugh
    str = re.sub(r"\b(?:(h|a|e)*(?:(ha|he|hue))+h?|(?:l+o+)+l+)|(?:(w|k)*(?:wk)+(w?|k?))\b", ' ', str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #remove digit or numbers
    str = re.sub(r"\b\d+\b", " ", str)
    #remove single character
    str = re.sub(r"\s+[a-zA-Z]\s+", " ", str) 
    #remove string
 #   str = re.sub('cc|nl|xa|xb|nhttps|nlink|nahy|admin|solopos|com', '', str)
    #remove_repeated_character
    str = re.sub(r'(.)\1{2,}', r'\1', str)
    #remove one character
    str = re.sub(r"\b[a-zA-Z]\b", "", str)
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
    #to lowercase
    str = str.lower()
    
    return str

def removeTwitterSymbols(str):
    #remove RT
    str = re.sub('RT', '', str)
    # hashtag
    str = re.sub(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", " ", str)
    #remove @username
    str = re.sub('@[^\s]+','',str)
     # mention
    str = re.sub(r'(?:@[\w_]+)', ' ', str)

    
    return str
    
def preprocessing(str):
    str = removeTwitterSymbols(str)
    str = denoise_text(str)
    str = cleaning(str)
    
    return str

In [11]:
def applyPreprocessing(file): 
    df = pd.read_excel(file)
    df['cleanText'] = df['text'].apply(preprocessing)
    df.assign(cleanText=df['cleanText'])
    label = file[:-5]
    df1 = df.assign(label=label)
    df1.dropna()
    writer = pd.ExcelWriter('%s-clean.xlsx'%(label)) #save to excel
    df1.to_excel(writer, 'Sheet1')
    writer.save()


file = 'Indo.xlsx'
file1 = 'Jawa.xlsx'
file2 = 'Sunda.xlsx'
applyPreprocessing("jawa.xlsx")

In [4]:
import xlsxwriter

In [9]:
#merge | menggabungkan seluruh data to one file excel
import glob
import pandas as pd

all_data = pd.DataFrame()
for f in glob.glob("data stabil/*.xlsx"): #perulangan untuk membuat dokumen baru berisikan semua data yang telah dipreprocessing
      df = pd.read_excel(f) #membaca file semua file yg dilakukan dengan perulangan
      df_new = df[['cleanText','label']] #membuat object baru berisikan data df
      all_data = all_data.append(df_new,ignore_index=True) #menggabungkan data object baru
    
writer = pd.ExcelWriter('indo-jawa-sunda stabil.xlsx', engine='xlsxwriter')
# Write your DataFrame to a file     
all_data.to_excel(writer, 'Sheet1')
# Save the result 
writer.save()  

In [ ]:
#sastrawi stopwords data

import Sastrawi

import os

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


def kata():
    dictionaryFile = 'google-10000-english.txt'
    if not os.path.isfile(dictionaryFile):
        raise RuntimeError('Dictionary file is missing. It seems that your installation is corrupted.')

    dictionaryContent = ''
    with open(dictionaryFile, 'r') as f:
        dictionaryContent = f.read()

    return dictionaryContent.split('\n')

kata = kata()

def hapus(file):
    #file = pd.read_excel(file)
    katatwitter = ['ahy','sby','xef','followers','follower','unfollower','unfollowers', 'unfollowed', 'xcb', 'xca', 'xcc', 'xba', 'xbb','xbe', 'xbd', 'xda', 'twit']
    stop_factory = StopWordRemoverFactory().get_stop_words()
    more_stopword = StemmerFactory().get_words_from_file()
    #data = stop_factory + kata + katatwitter
    data = stop_factory + more_stopword + kata + katatwitter
    #data = kata + katatiwtter
    dictionary = ArrayDictionary(data)
    str = StopWordRemover(dictionary)
    file = str.remove(file)
    return file

def try_model(filee):
    df11 = pd.read_excel(filee)
    df11['clean'] = df11['cleanText'].apply(hapus)
    df11.assign(text = df11['clean'])
    writer = pd.ExcelWriter('Sunda-stabil.xlsx')
    df11.to_excel(writer, 'Sheet1')
    writer.save()
    

In [ ]:
#try_model('Jawa-clean.xlsx')
try_model('Data new dan old digabung/Sunda Gabung.xlsx')